<a href="https://colab.research.google.com/github/upomadas-alt/Assignment_2_Python.ipynb/blob/main/Assignment_5_Data_Preprocessing_and_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [8]:
#Load the Data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TODO: Load the housing data
# Replace '/path/to/your/Housing.csv' with the actual path to your file
df = pd.read_csv('/content/drive/MyDrive/Datasets/Housing.csv')

# TODO: How many houses are there?
print(f"Number of houses: {len(df)}")  # Fill this

# TODO: Show first 3 houses
# Your code here
print("\nFirst 3 houses:")
print(df.head(3))

Number of houses: 545

First 3 houses:
      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  


In [10]:
# Look at Columns
# TODO: Show all column names
print("Columns:")
# Your code here
print(df.columns)

# TODO: Show data types of columns
print("Data types:")
print(df.dtypes)

Columns:
Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')
Data types:
price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


In [11]:
#Convert Yes/No to 1/0
# Check what's in our data
print("Current mainroad values:", df['mainroad'].head())
# Shows: yes, no, yes, yes, no

# Models need numbers, not text!
# We need: yes → 1, no → 0

# Convert Yes/No columns to 1/0
yes_no_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
for col in yes_no_cols:
    df[col] = df[col].apply(lambda x: 1 if x == 'yes' else 0)

# Display the changes (optional)
print("\nMainroad values after conversion:", df['mainroad'].head())

Current mainroad values: 0    yes
1    yes
2    yes
3    yes
4    yes
Name: mainroad, dtype: object

Mainroad values after conversion: 0    1
1    1
2    1
3    1
4    1
Name: mainroad, dtype: int64


In [13]:
# Split into Train and Test
# TODO: Split - 80% train, 20% test
df_train, df_test = train_test_split(df, train_size=0.8, random_state=100)

In [14]:
#Apply StandardScaler
# Area is huge (1650-13300), Bedrooms is small (1-6)!
#Your Code here

# Identify numerical columns (excluding the target variable 'price')
numerical_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols])
df_test[numerical_cols] = scaler.transform(df_test[numerical_cols])

# Display the scaled data (optional)
print("Scaled training data (first 5 rows):")
print(df_train.head())

print("\nScaled test data (first 5 rows):")
print(df_test.head())

Scaled training data (first 5 rows):
       price      area  bedrooms  bathrooms   stories  mainroad  guestroom  \
22   8645000  1.351873  0.078679  -0.579897 -0.929023         1          1   
41   7840000  0.565326  0.078679   1.410929  2.474792         1          0   
91   6419000  0.746837 -1.293483  -0.579897 -0.929023         1          1   
118  5950000  0.593251  0.078679  -0.579897 -0.929023         1          0   
472  3010000 -0.705249  1.450841  -0.579897  0.205582         1          0   

     basement  hotwaterheating  airconditioning   parking  prefarea  \
22          1                0                1  0.311484         0   
41          0                0                1 -0.849261         1   
91          1                0                0  1.472229         1   
118         1                0                1 -0.849261         1   
472         0                0                0  2.632974         0   

    furnishingstatus  
22         furnished  
41         furnished 

In [15]:
#train and test model
#your code here
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Separate features (X) and target (y)
X_train = df_train.drop('price', axis=1)
y_train = df_train['price']
X_test = df_test.drop('price', axis=1)
y_test = df_test['price']

# Handle categorical features ('furnishingstatus')
# We need to convert the 'furnishingstatus' column into numerical format using one-hot encoding.
X_train = pd.get_dummies(X_train, columns=['furnishingstatus'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['furnishingstatus'], drop_first=True)

# Ensure columns match between train and test after one-hot encoding
train_cols = X_train.columns
test_cols = X_test.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test[c] = 0
# Ensure the order of columns is the same
X_test = X_test[train_cols]


# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R-squared: {r2:.4f}")
print(f"Mean Squared Error: {mse:.2f}")

R-squared: 0.6807
Mean Squared Error: 1127125806714.59


In [16]:
# Predict a House Price
# New house details:
# area=5000, bedrooms=3, bathrooms=2, stories=2,
# mainroad=yes, guestroom=no, basement=yes,
# hotwaterheating=no, airconditioning=yes,
# parking=2, prefarea=yes, furnishingstatus=furnished
#your code here

# Create a DataFrame for the new house
new_house_data = {
    'area': [5000],
    'bedrooms': [3],
    'bathrooms': [2],
    'stories': [2],
    'mainroad': ['yes'],
    'guestroom': ['no'],
    'basement': ['yes'],
    'hotwaterheating': ['no'],
    'airconditioning': ['yes'],
    'parking': [2],
    'prefarea': ['yes'],
    'furnishingstatus': ['furnished']
}
new_house_df = pd.DataFrame(new_house_data)

# Apply the same preprocessing steps as the training data

# 1. Convert Yes/No to 1/0
yes_no_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
for col in yes_no_cols:
    new_house_df[col] = new_house_df[col].apply(lambda x: 1 if x == 'yes' else 0)

# 2. Apply StandardScaler to numerical columns
# Use the scaler fitted on the training data
numerical_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
new_house_df[numerical_cols] = scaler.transform(new_house_df[numerical_cols])

# 3. Handle categorical features ('furnishingstatus') using one-hot encoding
new_house_df = pd.get_dummies(new_house_df, columns=['furnishingstatus'], drop_first=True)

# Ensure the new house DataFrame has the same columns as the training data (X_train)
# Add missing columns with a value of 0
missing_cols = set(X_train.columns) - set(new_house_df.columns)
for c in missing_cols:
    new_house_df[c] = 0

# Ensure the order of columns is the same as X_train
new_house_df = new_house_df[X_train.columns]

# Predict the price
predicted_price = model.predict(new_house_df)

print(f"Predicted price for the new house: ${predicted_price[0]:,.2f}")

Predicted price for the new house: $7,335,725.10
